# Import

In [1]:
from google.colab import drive
drive.mount('/content/gdrive/', force_remount = True)

Mounted at /content/gdrive/


In [2]:
!pip -q install trax==1.3.1

     |████████████████████████████████| 348kB 5.9MB/s 
     |████████████████████████████████| 215kB 42.7MB/s 
     |████████████████████████████████| 1.5MB 15.2MB/s 
     |████████████████████████████████| 3.4MB 35.6MB/s 
     |████████████████████████████████| 368kB 56.8MB/s 
     |████████████████████████████████| 3.8MB 46.0MB/s 
     |████████████████████████████████| 1.9MB 45.1MB/s 
     |████████████████████████████████| 71kB 8.6MB/s 
     |████████████████████████████████| 1.2MB 44.7MB/s 
     |████████████████████████████████| 706kB 45.8MB/s 
     |████████████████████████████████| 194kB 49.8MB/s 
     |████████████████████████████████| 5.6MB 46.2MB/s 
     |████████████████████████████████| 655kB 51.6MB/s 
     |████████████████████████████████| 81kB 8.9MB/s 
     |████████████████████████████████| 358kB 46.3MB/s 
     |████████████████████████████████| 983kB 42.3MB/s 
     |████████████████████████████████| 368kB 43.3MB/s 
     |████████████████████████████████| 3.2MB 42.5MB/

In [52]:
import trax 
from trax import layers as tl
from trax.fastmath import numpy as fastnp
from trax.supervised import training
import os 
import numpy as np
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('stopwords')
from gdrive.MyDrive.TP_11.utils import get_params, get_vocab
import random as rnd
import tensorflow as tf
from tensorflow import keras

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Récupération des données

In [4]:
data = pd.read_csv("gdrive/MyDrive/ProjetNLP/training.csv", encoding = "ISO-8859-1", header=None) # Récupération des données
data.head()

,0,1,2,3,4,5
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [5]:
data_test = data[799000:801000] # Création d'un jeu de données de test
data_test

,0,1,2,3,4,5
799000,0,2328818606,Thu Jun 25 10:01:34 PDT 2009,NO_QUERY,redstar72,"CAN'T BEAT LIVE MUSIC, WISH I COULD SING BUT I..."
799001,0,2328818761,Thu Jun 25 10:01:35 PDT 2009,NO_QUERY,LIDA360,Charlie lost an angel today Very sad http:...
799002,0,2328818847,Thu Jun 25 10:01:35 PDT 2009,NO_QUERY,rnspires21,"at work, and very bored"
799003,0,2328819310,Thu Jun 25 10:01:37 PDT 2009,NO_QUERY,lasthonestlook,It's weird how celebrities go in threes. Carra...
799004,0,2328819347,Thu Jun 25 10:01:37 PDT 2009,NO_QUERY,deiann,@adelate Farrah's dead? Had no idea.. RIP Farr...
...,...,...,...,...,...,...
800995,4,1468005450,Mon Apr 06 23:13:13 PDT 2009,NO_QUERY,KiwiBeck,I have this strange desire to go to confession...
800996,4,1468005505,Mon Apr 06 23:13:14 PDT 2009,NO_QUERY,astrologee,@i_reporter answer sent in dm. try it
800997,4,1468005511,Mon Apr 06 23:13:16 PDT 2009,NO_QUERY,Melamachinko,@brooklynunion cuz ur 3pm is my 9am and Id be ...
800998,4,1468005581,Mon Apr 06 23:13:16 PDT 2009,NO_QUERY,Yahtzee27,@littrellfans Its all good. Just figured you w...


# Création résulats attendus

In [6]:
results = []                   # Création d'une liste contenant la valeur des données testées. 0 si négatif, 1 si positif
for i in data_test.iloc(0):
  results.append(int(i[0]))

for i in range(len(results)):
  if results[i] == 4:
    results[i] = 1
results

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


# Préparation des données

In [7]:
tweet = []
tokenizer = nltk.RegexpTokenizer(r'\w+')
english_stopwords = nltk.corpus.stopwords.words("english")
for i in data_test.iloc(0):
  new_tweet = ""
  new_tweet += i[5]
  tweet.append(tokenizer.tokenize(new_tweet.lower()))  # Toenization des données


tweet_sw = []
for i in tweet:
  new_tweet_sw = ""
  for j in i:
    if j not in english_stopwords:
      new_tweet_sw += (j + " ")
  tweet_sw.append(tokenizer.tokenize(new_tweet_sw))    # Suppression des stopwords

tweet_sw

[['beat', 'live', 'music', 'wish', 'could', 'sing'],
 ['charlie', 'lost', 'angel', 'today', 'sad', 'http', 'twurl', 'nl', 'k5bw5m'],
 ['work', 'bored'],
 ['weird', 'celebrities', 'go', 'threes', 'carradine', 'mcmahon', 'farrah'],
 ['adelate', 'farrah', 'dead', 'idea', 'rip', 'farrah', 'indeed'],
 ['sad', 'news', 'farrah', 'fawcett', 'passed', 'away'],
 ['sad',
  'hear',
  'news',
  'farrah',
  'fawcett',
  'corpus',
  'christi',
  'native',
  'missed',
  'dearly'],
 ['r', 'p', 'farrah', 'fawcett', 'super', 'hottie', 'end', 'real', 'angel'],
 ['damn', 'plans', '4', '2morrow', 'r', 'cancelled', 'wat', 'supposed'],
 ['rest', 'peace', 'farrah', 'fawcett', 'sad'],
 ['retweeting',
  'secretagentmama',
  'rest',
  'peace',
  'farrah',
  'fawcett',
  'http',
  'gd',
  '1dfib'],
 ['yay',
  'haruhi',
  'endless',
  'eight',
  'part',
  '2',
  '3',
  'thought',
  'two',
  'episodes',
  'many'],
 ['really',
  'need',
  'keep',
  'parts',
  'help',
  'find',
  'cure',
  'cancer',
  'many',
  'lives

In [8]:
max_len = 0                 # Création d'une variable contenant la taille du plus grand tweet testé 
for tweet in tweet_sw:
  if len(tweet) > max_len:
    max_len = len(tweet)

max_len

24

# Création du vocabulaire

In [9]:
from collections import defaultdict

vocab = defaultdict(lambda: 0)
vocab['<PAD>'] = 1

for tweet in tweet_sw:
  for word in tweet:
    if word not in vocab:
      vocab[word] = len(vocab) + 1      # Création du vocabulaire

print(vocab)

defaultdict(<function <lambda> at 0x7f33fbc9bd40>, {'<PAD>': 1, 'beat': 2, 'live': 3, 'music': 4, 'wish': 5, 'could': 6, 'sing': 7, 'charlie': 8, 'lost': 9, 'angel': 10, 'today': 11, 'sad': 12, 'http': 13, 'twurl': 14, 'nl': 15, 'k5bw5m': 16, 'work': 17, 'bored': 18, 'weird': 19, 'celebrities': 20, 'go': 21, 'threes': 22, 'carradine': 23, 'mcmahon': 24, 'farrah': 25, 'adelate': 26, 'dead': 27, 'idea': 28, 'rip': 29, 'indeed': 30, 'news': 31, 'fawcett': 32, 'passed': 33, 'away': 34, 'hear': 35, 'corpus': 36, 'christi': 37, 'native': 38, 'missed': 39, 'dearly': 40, 'r': 41, 'p': 42, 'super': 43, 'hottie': 44, 'end': 45, 'real': 46, 'damn': 47, 'plans': 48, '4': 49, '2morrow': 50, 'cancelled': 51, 'wat': 52, 'supposed': 53, 'rest': 54, 'peace': 55, 'retweeting': 56, 'secretagentmama': 57, 'gd': 58, '1dfib': 59, 'yay': 60, 'haruhi': 61, 'endless': 62, 'eight': 63, 'part': 64, '2': 65, '3': 66, 'thought': 67, 'two': 68, 'episodes': 69, 'many': 70, 'really': 71, 'need': 72, 'keep': 73, 'part

# Padding

In [10]:
tweets_p = []                          # Padding

for tweet in tweet_sw:
  tweet_p = []
  for i in range(max_len):
    tweet_p.append(vocab['<PAD>'])
  for i in range(len(tweet)):
    tweet_p[i] = vocab[tweet[i]]
  tweets_p.append(tweet_p)
print(tweets_p)

[[2, 3, 4, 5, 6, 7, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [8, 9, 10, 11, 12, 13, 14, 15, 16, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [17, 18, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [19, 20, 21, 22, 23, 24, 25, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [26, 25, 27, 28, 29, 25, 30, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [12, 31, 25, 32, 33, 34, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [12, 35, 31, 25, 32, 36, 37, 38, 39, 40, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [41, 42, 25, 32, 43, 44, 45, 46, 10, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [47, 48, 49, 50, 41, 51, 52, 53, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [54, 55, 25, 32, 12, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [56, 57, 54, 55, 25, 32, 13, 58, 59, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [71, 72, 73, 74, 75, 76, 77, 7

# Modèle

In [59]:
# Création du modèle
model = tf.keras.models.Sequential()

model.add(tf.keras.layers.Embedding(input_dim = 10000, output_dim = 10000))
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(15)))

model.add(tf.keras.layers.Dense(6, activation = 'relu'))
model.add(tf.keras.layers.Dense(1, activation = 'sigmoid'))

# Entrainement

In [13]:
model.compile(optimizer= tf.keras.optimizers.Adam(0.01), loss='binary_crossentropy', metrics=['accuracy'])
model.fit(tweets_p,results,epochs = 10) # Entrainement du modèle

Epoch 1/10
63/63 [==============================] - 87s 1s/step - loss: 0.6096 - accuracy: 0.6439
Epoch 2/10
63/63 [==============================] - 83s 1s/step - loss: 0.2588 - accuracy: 0.9140
Epoch 3/10
63/63 [==============================] - 82s 1s/step - loss: 0.1235 - accuracy: 0.9584
Epoch 4/10
63/63 [==============================] - 82s 1s/step - loss: 0.1030 - accuracy: 0.9626
Epoch 5/10
63/63 [==============================] - 85s 1s/step - loss: 0.0850 - accuracy: 0.9698
Epoch 6/10
63/63 [==============================] - 89s 1s/step - loss: 0.0775 - accuracy: 0.9700
Epoch 7/10
63/63 [==============================] - 83s 1s/step - loss: 0.0926 - accuracy: 0.9637
Epoch 8/10
63/63 [==============================] - 82s 1s/step - loss: 0.0777 - accuracy: 0.9654
Epoch 9/10
63/63 [==============================] - 82s 1s/step - loss: 0.0760 - accuracy: 0.9662
Epoch 10/10
63/63 [==============================] - 82s 1s/step - loss: 0.0680 - accuracy: 0.9708


In [45]:
model.save("gdrive/MyDrive/ProjetNLP/model")    # Sauvegarde du modèle

INFO:tensorflow:Assets written to: gdrive/MyDrive/ProjetNLP/model/assets


INFO:tensorflow:Assets written to: gdrive/MyDrive/ProjetNLP/model/assets


In [37]:
model.predict(np.expand_dims(tweets_p[90], axis=0))  #Test concluant sur un tweet négatif

array([[0.02146098]], dtype=float32)

# Pipeline

In [56]:
def reponse(phrase):
  model1 = keras.models.load_model('gdrive/MyDrive/ProjetNLP/model/')  # Récupération du modèle
  phrase = tokenizer.tokenize(phrase)
  phrase_sw = []
  for word in phrase:
    if word not in english_stopwords:
      phrase_sw.append(word)

  phrase_pd = []
  for word in phrase_sw:
    if word in vocab:
      phrase_pd.append(vocab[word])
    else:
      phrase_pd.append(0)
  
  if len(phrase_pd) < max_len:
    for i in range(max_len - len(phrase_pd)):
      phrase_pd.append(vocab['<PAD>'])

  answer = model1.predict(np.expand_dims(phrase_pd, axis=0))

  if answer > 0.5:  # Gestion réponse positive ou négative selon le retour du modèle.predict
    return "Positif"
  else:
    return "Négatif"

# Test

In [57]:
reponse("I am feeling really good")

'Positif'

In [58]:
reponse("I am feeling really bad")

'Négatif'